<p style="text-align: right">
  <!--<img src="../images/dtlogo.png" alt="Logo" width="200">-->
</p>

# 💻 02 - Stereo Calibration
## 1 : Intrinsics calibration

We created a wrapper around the official [tutorial](https://docs.luxonis.com/en/latest/pages/calibration/) in the Luxonis Documentation. <br>
To calibrate the camera you need to print the calibration board:
<figure>
  <div style="text-align:center;">
  <img src="../images/calib_board.png">
  </div>
</figure>


To calibrate the device you can plug it to the computer and run: <br><br>
`dts oakd calibrate`<br><br>
The Luxonis calibration tool will start, please follow the instruction and procede to the calibration.

## 2 : Extrinsics calibration

Now that we have the intrisic parameters, we can procede to compute the extrinsics paramters. Namely, the transformation from the road to the camera frame.<br>
You can follo the same steps described in the Duckietown camera calibration [tutorial](https://docs.duckietown.org/daffy/opmanual_duckiebot/out/camera_calib.html) in the section 11.3.<br>


⚠️ ***WARNING:***
- MAKE SURE YOU ARE RUNNING THE BASIC ACTIVITY FIRST
- MAKE SURE THE CAMERA IS STREAMING THE IMAGES UNDER THE TOPIC `/DB_NAME/camera_node/image/compressed`

### 💡 Next steps

Go back in the basic tutorial, after the calibration how is the disparoty map changed?
You can now proceed to the next activity: [Semantic Segmentation](../03-semantic-segmentation/semantic_segmentation.ipynb).